In [ ]:
!pip install lightgbm
!pip install surprise
!pip install -U mlxtend

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#
import mlxtend
from surprise import Reader, Dataset, accuracy, SVD
#from surprise.model_selection import train_test_split, GridSearchCV
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def transObj(df):
  le = LabelEncoder()
  for x in df:
      if df[x].dtypes=='object':
          df[x] = le.fit_transform(df[x])
  return df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 1: 보험 가격 예측

# 결과
# RMSE: 1579.5409127178361

Mm = pd.read_csv("Medicalpremium.csv");

y = Mm.PremiumPrice
X = Mm.drop("PremiumPrice", axis=1)

# 1_1 전처리
# 1_1_1 결측치 처리

#print(Mm.isnull().sum() / Mm.isna().sum())
#결과: 없음 생략

# 1_2 스케일링
Mm_scaled = MinMaxScaler().fit_transform( X )

# 1_3 파티셔닝
X_train, X_test, y_train, y_test = train_test_split( Mm_scaled, y, test_size=0.2)

# 1_4 fit
# 1_4_1 모델 및 파라메타 선정
# 모델: MLA 확인시  LGBMClassifier가 가장 좋다고 나옴
# 파라메타: 디폴트가 제일 좋다고 나옴 0.1 / 100

# 파라메타 검색
# lgbm_params = {"learning_rate": [0.1, 0.3, 0.5, 0.7,1.0], "n_estimators":[100,200,300]}
# c2_lgbm = GridSearchCV(LGBMClassifier(), lgbm_params).fit(X_train, y_train)
# c2_lgbm.best_params_

model = LGBMClassifier().fit( X_train, y_train )

# 1_5 RMSE
print("RMSE - ", np.sqrt( np.mean( (model.predict( X_test) - y_test)**2 ) ))

1.0
RMSE -  1579.5409127178361


In [ ]:
# 2. 신용카드 승인 분류

# 결과
# 정확도 accuracy: 0.992
# 정밀도 precision: 0.550
# 재현율 recall: 0.177


# status 설명
# 0: 1-29일 연체
# 1: 30-59일 연체
# 2: 60-89일 연체
# 3: 90-119일 연체
# 4: 120-149일 연체
# 5: 연체 또는 부실 채권, 150일 이상 상각
# C: 그 달에 상환
# X: 해당 월에 대출 없음

ar = pd.read_csv("application_record.csv")
cr = pd.read_csv("credit_record.csv")

# 2_1 전처리
# 2_1_1 중복제거
# print(ar['ID'].nunique())
ar = ar.drop_duplicates('ID', keep='last')

# 2_1_2 결측 OCCUPATION_TYPE에만 있음
ar['OCCUPATION_TYPE'] = ar['OCCUPATION_TYPE'].fillna('others')

# 2_1_3 Object 값을 라벨링
le = LabelEncoder()
transObj(ar)


# 2_1_4 status 변경
cr['STATUS'].replace({'C': 0, 'X': 0, '0' : 0, '1': 0, '2': 0}, inplace=True)
cr['STATUS'].replace({'3': 1, '4': 1, '5' : 1}, inplace=True)
ar['DAYS_EMPLOYED'] = ar['DAYS_EMPLOYED'].apply(lambda x:1 if x > 0 else 0)

# 2_1_5 group by 및 df 합치기
crGroup = cr.groupby('ID').agg(max).reset_index()
data = ar.join(crGroup.set_index('ID'), on='ID', how='inner')

y = data.STATUS
X = data.drop("STATUS", axis=1)

# 2_2 스케일링
ar_scaled = MinMaxScaler().fit_transform( X )

# 2_3 파티셔닝
X_train, X_test, y_train, y_test = train_test_split( ar_scaled, y, test_size=0.2)

# 2_4 오버샘플링 안하는게 나음
oversample = SMOTE(sampling_strategy = 1 ,k_neighbors = 1)
xTrainB, yTrainB = oversample.fit_resample(X_train, y_train)

model1 = LGBMClassifier().fit( X_train, y_train)
y_pred1 = model1.predict(X_test)

print('정확도 accuracy: %.3f' % accuracy_score(y_test, y_pred1))
print('정밀도 precision: %.3f' % precision_score(y_true= y_test, y_pred=y_pred1))
print('재현율 recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred1))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-49-d2b275ae7e76>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ar['OCCUPATION_TYPE'] = ar['OCCUPATION_TYPE'].fillna('others')
<ipython-input-29-922f656a3aa6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

정확도 accuracy: 0.992
정밀도 precision: 0.565
재현율 recall: 0.217


In [ ]:
# 3. 은행 마케팅

# 결과
# 정확도 accuracy: 0.901
# 정밀도 precision: 0.624
# 재현율 recall: 0.428

bf = pd.read_csv("bank-full.csv")
transObj(bf)

y = bf.y
X = bf.drop("y", axis=1)

# 3_1 전처리
# 3_1_1 결측치 없음

# 3_2 스케일링
bf_scaled = MinMaxScaler().fit_transform( X )

# 3_3 파티셔닝
X_train, X_test, y_train, y_test = train_test_split(bf_scaled, y, test_size=0.2)

model = RandomForestClassifier().fit( X_train, y_train)
y_pred = model.predict(X_test)

print('정확도 accuracy: %.3f' % accuracy_score(y_test, y_pred))
print('정밀도 precision: %.3f' % precision_score(y_true= y_test, y_pred=y_pred))
print('재현율 recall: %.3f' % recall_score(y_true=y_test, y_pred=y_pred))

정확도 accuracy: 0.901
정밀도 precision: 0.624
재현율 recall: 0.428


In [ ]:
#RMSE: 3.4598
#RMSE: 3.4612
book = pd.read_csv("BX-Book-Ratings._utf8.csv", sep=";")
book = book.dropna(how = 'any')

reader = Reader(rating_scale=(0.0, 10.0))
data = Dataset.load_from_df(book[['User-ID', 'ISBN', 'Book-Rating']], reader)

train, test = train_test_split(data)

algo = SVD(n_epochs=60, n_factors=300)
algo.fit(train)
pred = algo.test( test )
accuracy.rmse(pred)

algo = SVD(n_epochs=40, n_factors=300)
algo.fit(train)
pred = algo.test( test )
accuracy.rmse(pred)

RMSE: 3.4449
RMSE: 3.4481


3.4481088240608577

In [ ]:
# ID	ITEM	TIMESTAMP
m1 = pd.read_csv("Behance_appreciate_1M", header=None, sep=" ", names=["ID","ITEM","TIMESTAMP"])

m1Group = m1.groupby('ID')['ITEM'].apply(set).apply(list).to_list()

te = TransactionEncoder()
te_ary = te.fit_transform(m1Group)

df = pd.DataFrame(te_ary, columns=te.columns_)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
frequent_itemsets = apriori(df, min_support=0.005, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False)

ar1 = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1).sort_values(by = ['confidence', 'lift', 'support'], ascending=False)
ar2 = association_rules(frequent_itemsets, metric="lift", min_threshold=1).sort_values(by = [ 'lift', 'confidence', 'support'], ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(ar1)
print(ar2)

   antecedents  consequents  antecedent support  consequent support   support  \
0  (1489915.0)  (1316541.0)            0.013791            0.029017  0.006212   
1  (1316541.0)  (1489915.0)            0.029017            0.013791  0.006212   

   confidence       lift  leverage  conviction  zhangs_metric  
0    0.450407  15.521969  0.005811    1.766729       0.948658  
1    0.214065  15.521969  0.005811    1.254822       0.963534  
   antecedents  consequents  antecedent support  consequent support   support  \
0  (1489915.0)  (1316541.0)            0.013791            0.029017  0.006212   
1  (1316541.0)  (1489915.0)            0.029017            0.013791  0.006212   

   confidence       lift  leverage  conviction  zhangs_metric  
0    0.450407  15.521969  0.005811    1.766729       0.948658  
1    0.214065  15.521969  0.005811    1.254822       0.963534  


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
frequent_itemsets2 = fpgrowth(df, min_support=0.005, use_colnames=True)
frequent_itemsets2.sort_values(by='support', ascending=False)

fr1 = association_rules(frequent_itemsets2, metric="confidence", min_threshold=0.1).sort_values(by = ['confidence', 'lift', 'support'], ascending=False)
fr2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1).sort_values(by = [ 'lift', 'confidence', 'support'], ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(fr1)
print(fr2)

   antecedents  consequents  antecedent support  consequent support   support  \
0  (1489915.0)  (1316541.0)            0.013791            0.029017  0.006212   
1  (1316541.0)  (1489915.0)            0.029017            0.013791  0.006212   

   confidence       lift  leverage  conviction  zhangs_metric  
0    0.450407  15.521969  0.005811    1.766729       0.948658  
1    0.214065  15.521969  0.005811    1.254822       0.963534  
   antecedents  consequents  antecedent support  consequent support   support  \
0  (1489915.0)  (1316541.0)            0.013791            0.029017  0.006212   
1  (1316541.0)  (1489915.0)            0.029017            0.013791  0.006212   

   confidence       lift  leverage  conviction  zhangs_metric  
0    0.450407  15.521969  0.005811    1.766729       0.948658  
1    0.214065  15.521969  0.005811    1.254822       0.963534  


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

X = X_train
y = y_train

#다양한 분류모형 비교
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

MLA = [
    #Tree, Ensemble Methods
    tree.DecisionTreeClassifier(),
    XGBClassifier(),
    LGBMClassifier(),
    ensemble.AdaBoostClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #GLM
    linear_model.LogisticRegressionCV(),

    #Nearest Neighbor
    KNeighborsClassifier()]
#http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit

#train_test_split보다 편하게 여러개 생성 가능 (10개를 만들거임 60:30으로 10은 뺴두기)
cv_split = model_selection.ShuffleSplit(n_splits = 5, test_size = .3, train_size = .6, random_state = 0 ) # run model 10x with 60/30 split intentionally leaving out 10%

#MLA 비교
MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)

#create table to compare MLA predictions
MLA_predict = pd.DataFrame(y)

#index through MLA and save performance to table
row_index = 0
for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())

    #CV score: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate
    cv_results = model_selection.cross_validate(alg, X, y, cv  = cv_split,return_train_score=True)

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
    MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!

    #save MLA predictions
    alg.fit(X, y)
    MLA_predict[MLA_name] = alg.predict(X)
    row_index+=1


MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
MLA_compare